In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy 
import os
import unittest

## Battery Dataset Full

In [20]:
#Upload metadata
metadata = pd.read_csv('snl_metadata_cycle_500.csv')

#Define data directory and load files, ensure the data directory and this notebook are in the same directory
data_dir = "Cycle Data"
files = os.listdir(data_dir)

#Initialize dataframe header row 
df = pd.read_csv(data_dir+"/"+files[0])
df_output = df.head(0)

# Define cycles 
cycle = [50.0, 100.0, 150.0, 200.0, 250.0, 300.0, 350.0, 400.0, 450.0, 500.0]

  

In [21]:
#Build dataframe by adding the correct row of data pulled from "Cycle" files
#Include the output "Coulombic efficiency"
#Add the cell_id from "Cycle" files

cell_id_2 =[]

i=0                
for file in files:
    df = pd.read_csv(data_dir+"/"+files[i])
    
    for j in range(len(cycle)):
        df_cycles = df.loc[df['Cycle_Index'] == cycle[j]]
        df_output = df_output.append(df_cycles, ignore_index = True)
        
        df_output['Coulombic_Efficiency (%)'] = df_output['Discharge_Capacity (Ah)']/df_output['Charge_Capacity (Ah)']*100
        df_output['Energy_Efficiency (%)'] = df_output['Discharge_Energy (Wh)']/df_output['Charge_Energy (Wh)']*100
        
        cell_id_2.append(file[:-15])
        df_output['cell_id_2'] = cell_id_2
        
    i+=1     

In [22]:
df_output = pd.concat([metadata, df_output],axis=1)

In [23]:
#Check to make sure data pulled from "Cycle" files match the correct metadata line
for i in range(len(df_output)):
    if df_output['cell_id'][i] == df_output['cell_id_2'][i]:
        pass
    else:
        print('not a match', df_output['cell_id'][i],df_output['cell_id_2'][i])
        
for i in range(len(df_output)):
    if df_output['Cycle'][i] == df_output['Cycle_Index'][i]:
        pass
    else:
        print('not a match', df_output['Cycle'][i],df_output['Cycle_Index'][i])

In [24]:
#Load full Battery dataset
df_battery_full = df_output
df_battery_full = df_battery_full.drop('cell_id_2', 1)
df_battery_full = df_battery_full.drop('Cycle_Index', 1)

/tmp/ipykernel_425/3032663218.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_battery_full = df_battery_full.drop('cell_id_2', 1)
/tmp/ipykernel_425/3032663218.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_battery_full = df_battery_full.drop('Cycle_Index', 1)


In [25]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# df_battery_full

## Battery Dataset 

In [26]:
#Create a column that will indicated replicated experiments 
df_battery_full['rep'] = None 

#Populate the column, replicates will have a value of 1
for i in range(len(df_battery_full)):
    if df_battery_full.cell_id.values[i][-1] == 'a':
        df_battery_full['rep'][i] = 0
    if df_battery_full.cell_id.values[i][-1] != 'a':
        df_battery_full['rep'][i] = 1

#Re

/tmp/ipykernel_425/492022663.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_battery_full['rep'][i] = 0
/tmp/ipykernel_425/492022663.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_battery_full['rep'][i] = 1


In [27]:
#Drop any row of replicated experiment data and load filtered Battery Dataset
df_battery = df_battery_full[df_battery_full['rep'] < 0.5]
df_battery = df_battery.drop('rep', 1)
df_battery = df_battery.reset_index(drop=True)

/tmp/ipykernel_425/1923714681.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_battery = df_battery.drop('rep', 1)


In [28]:
df_battery


,cell_id,study,Cycles,anode,cathode,electrolyte,Cycle,temperature,min_soc,max_soc,...,Min_Current (A),Max_Current (A),Min_Voltage (V),Max_Voltage (V),Charge_Capacity (Ah),Discharge_Capacity (Ah),Charge_Energy (Wh),Discharge_Energy (Wh),Coulombic_Efficiency (%),Energy_Efficiency (%)
0,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",graphite,LFP,A123,50,15,0,100,...,-1.1,0.55,1.998,3.6,1.036,1.033,3.495,3.213,99.710425,91.931330
1,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",graphite,LFP,A123,100,15,0,100,...,-1.1,0.55,1.996,3.6,1.035,1.032,3.492,3.212,99.710145,91.981672
2,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",graphite,LFP,A123,150,15,0,100,...,-1.1,0.55,1.998,3.6,1.034,1.031,3.488,3.210,99.709865,92.029817
3,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",graphite,LFP,A123,200,15,0,100,...,-1.1,0.55,1.994,3.6,1.033,1.030,3.490,3.193,99.709584,91.489971
4,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",graphite,LFP,A123,250,15,0,100,...,-1.1,0.55,1.998,3.6,1.031,1.028,3.495,3.169,99.709020,90.672389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,graphite,NMC,LGC,300,35,0,100,...,-6.0,1.50,1.996,4.2,2.529,2.527,9.814,8.388,99.920917,85.469737
316,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,graphite,NMC,LGC,350,35,0,100,...,-6.0,1.50,1.994,4.2,2.518,2.506,9.769,8.332,99.523431,85.290204
317,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,graphite,NMC,LGC,400,35,0,100,...,-6.0,1.50,1.995,4.2,2.484,2.482,9.650,8.197,99.919485,84.943005
318,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,graphite,NMC,LGC,450,35,0,100,...,-6.0,1.50,1.995,4.2,2.477,2.478,9.617,8.208,100.040371,85.348861


In [29]:
df_battery.to_csv('Battery_Dataset.csv', index=False)

In [37]:
X = df_battery.loc[:,['anode','cathode','electrolyte']]
Y = df_battery.loc[:,['cathode']]
Z = df_battery.loc[:,['electrolyte']]

In [38]:
ohe = OneHotEncoder()
X = ohe.fit_transform(X)
Y = ohe.fit_transform(Y)
Z = ohe.fit_transform(Z)

In [39]:
xx = X.toarray()
yy = Y.toarray()
zz = Z.toarray()

In [46]:
xx[300]

array([1., 0., 0., 1., 0., 1., 0.])